<a href="https://colab.research.google.com/github/JoshStrother/DataVisualization/blob/main/lesson12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Firstly we authenticated and now I am going to just look at the data that I actually have.

In [2]:
%%bigquery --project data-viz-338716
SELECT *
FROM `bigquery-public-data.london_fire_brigade.fire_brigade_service_calls`
LIMIT 5

,incident_number,date_of_call,cal_year,time_of_call,hour_of_call,timestamp_of_call,incident_group,stop_code_description,special_service_type,property_category,property_type,address_qualifier,postcode_full,postcode_district,borough_code,borough_name,proper_case,ward_code,ward_name,ward_name_new,easting_m,northing_m,easting_rounded,northing_rounded,frs,incident_station_ground,first_pump_arriving_attendance_time,first_pump_arriving_deployed_from_station,second_pump_arriving_attendance_time,second_pump_arriving_deployed_from_station,num_stations_with_pumps_attending,num_pumps_attending
0,000320-01012017,2017-01-01,2017,15:30:22,15,2017-01-01 15:30:22+00:00,False Alarm,AFA,None,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Within same building,None,IG9,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,None,None,541450,194050,OverTheBorder,OverTheBorder,None,None,None,None,2,2
1,003727-09012017,2017-01-09,2017,10:41:54,10,2017-01-09 10:41:54+00:00,False Alarm,AFA,None,Dwelling,House - single occupancy,Correct incident location,None,IG7,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,None,None,543650,192950,OverTheBorder,OverTheBorder,None,None,None,None,2,2
2,003762-09012017,2017-01-09,2017,12:28:39,12,2017-01-09 12:28:39+00:00,False Alarm,AFA,None,Dwelling,House - single occupancy,Correct incident location,None,IG7,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,None,None,543650,192950,OverTheBorder,OverTheBorder,None,None,None,None,2,2
3,006403-16012017,2017-01-16,2017,04:02:14,4,2017-01-16 04:02:14+00:00,False Alarm,AFA,None,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Correct incident location,None,IG7,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,None,None,544350,192250,OverTheBorder,OverTheBorder,None,None,None,None,2,2
4,010947-26012017,2017-01-26,2017,11:14:03,11,2017-01-26 11:14:03+00:00,False Alarm,AFA,None,Dwelling,House - single occupancy,Correct incident location,None,IG10,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,None,None,542050,196450,OverTheBorder,OverTheBorder,None,None,None,None,1,1


Okay now we want to count the number of rows in order to answer 1.A

In [5]:
%%bigquery --project data-viz-338716
SELECT COUNT(*) as num_of_rows
FROM `bigquery-public-data.london_fire_brigade.fire_brigade_service_calls`

,num_of_rows
0,32247


Now we want to count the number of false alarms between 10PM and 5AM, which in the data would be >= 22 and <= 5.

In [10]:
%%bigquery --project data-viz-338716
SELECT COUNT(*) as num_of_false_alarms_late_night
FROM `bigquery-public-data.london_fire_brigade.fire_brigade_service_calls`
WHERE incident_group = "False Alarm" AND (hour_of_call >= 22 OR hour_of_call <= 5) 

,num_of_false_alarms_late_night
0,3098


Next we want to get the AVG and in order to be more specific we will group based on the incident.

In [14]:
%%bigquery --project data-viz-338716
SELECT incident_group as fire_type, AVG(num_pumps_attending) as num_of_pumps
FROM `bigquery-public-data.london_fire_brigade.fire_brigade_service_calls`
GROUP BY incident_group

,fire_type,num_of_pumps
0,False Alarm,1.725776
1,Fire,1.633012
2,Special Service,1.177192


Now we want the most number of pumps used as a Road Vehicle property type.  So we will gather the max num of pumps and the property, then we will narrow it down to when its a Road Vehicle and groupby the property type.

In [17]:
%%bigquery --project data-viz-338716
SELECT property_category, Max(num_pumps_attending) as num_of_pumps
FROM `bigquery-public-data.london_fire_brigade.fire_brigade_service_calls`
WHERE property_category = "Road Vehicle"
GROUP BY property_category

,property_category,num_of_pumps
0,Road Vehicle,6


----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
import requests
import pandas as pa
from bs4 import BeautifulSoup


r = requests.get('https://en.wikipedia.org/wiki/Oklahoma')
html_contents = r.text
html_soup = BeautifulSoup(html_contents,"lxml")

In [20]:
tables = html_soup.find_all('table',class_="wikitable")
tables[0].find_all('a')

[<a href="#cite_note-99">[98]</a>, <a href="#cite_note-100">[99]</a>]

In [21]:
df = pa.read_html(str(tables))[0]
df

,City,Unnamed: 1,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,Oklahoma City,Avg. high,50 °F(10 °C),55 °F(13 °C),63 °F(17 °C),73 °F(23 °C),80 °F(27 °C),88 °F(31 °C),94 °F(34 °C),93 °F(34 °C),85 °F(29 °C),73 °F(23 °C),62 °F(17 °C),51 °F(11 °C)
1,Oklahoma City,Avg. low,29 °F(−2 °C),33 °F(1 °C),41 °F(5 °C),50 °F(10 °C),60 °F(16 °C),68 °F(20 °C),72 °F(22 °C),71 °F(22 °C),63 °F(17 °C),52 °F(11 °C),40 °F(4 °C),31 °F(−1 °C)
2,Tulsa,Avg. high,48 °F(9 °C),53 °F(12 °C),62 °F(17 °C),72 °F(22 °C),79 °F(26 °C),88 °F(31 °C),93 °F(34 °C),93 °F(34 °C),84 °F(29 °C),73 °F(23 °C),61 °F(16 °C),49 °F(9 °C)
3,Tulsa,Avg. low,27 °F(−3 °C),31 °F(−1 °C),40 °F(4 °C),49 °F(9 °C),59 °F(15 °C),68 °F(20 °C),73 °F(23 °C),71 °F(22 °C),62 °F(17 °C),51 °F(11 °C),40 °F(4 °C),30 °F(−1 °C)
4,Lawton,Avg. high,50 °F(10 °C),56 °F(13 °C),65 °F(18 °C),73 °F(23 °C),82 °F(28 °C),90 °F(32 °C),96 °F(36 °C),95 °F(35 °C),86 °F(30 °C),76 °F(24 °C),62 °F(17 °C),52 °F(11 °C)
5,Lawton,Avg. low,26 °F(−3 °C),31 °F(−1 °C),40 °F(4 °C),49 °F(9 °C),59 °F(15 °C),68 °F(20 °C),73 °F(23 °C),71 °F(22 °C),63 °F(17 °C),51 °F(11 °C),39 °F(4 °C),30 °F(−1 °C)
